In [ ]:
import pandas as pd
from collections import Counter

In [ ]:
data = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/te-samples',sep='\t')

users = Counter(data['user'])

selected = [u for u,v in users.items() if v>=10]

selected_data = data[data['user'].isin(selected)].groupby('user').sample(10)

selected_data.to_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/amazon_test_samples',sep='\t',index=None)

In [ ]:
data = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/reddit_test',sep='\t')

users = Counter(data['user'])
selected = [u for u,v in users.items() if v>=10]
len(selected)

selected_data = data[data['user'].isin(selected)].groupby('user').sample(10)

selected_data.to_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/reddit_test_samples',sep='\t',index=None)

In [ ]:
import seaborn as sns
import pandas as pd
import torch
import transformers
import numpy as np
import gensim
from collections import defaultdict

from torch import nn
import torch.nn.functional as F
from transformers import RobertaConfig, BertTokenizerFast, BertModel, RobertaTokenizer, RobertaModel, AdamW,RobertaTokenizerFast
from tqdm import tqdm

from model_amazon_self_attention import SRoberta,DNNSelfAttention,AttentionPooling

In [ ]:
device = 'cpu'

def get_conicity(vectors):
    mean_vec = torch.mean(vectors,dim=0)
    mean_vec = F.normalize(mean_vec,dim=-1).unsqueeze(-1)
    atm = torch.matmul(vectors,mean_vec)
    conicity = torch.mean(atm)
    return atm,conicity


def extract_emb(text):
    tokenized = tokenizer.encode_plus(text,add_special_tokens=True, max_length=102,truncation=True,return_tensors="pt")
    hidden = model(tokenized['input_ids'],tokenized['attention_mask'])
    hidden = hidden
    hidden = F.normalize(hidden,dim=-1)
#    hidden = hidden.cpu().detach().numpy()
    return hidden

In [ ]:
model_path = '/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship_models/final-roberta-cosine-modified_anchor-mask-0.1-delta-0.4-0.6-alpha-30.0/model-5'
model = torch.load(model_path,map_location=torch.device('cpu'))
model.eval()
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

In [ ]:
data = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/amazon_test_samples',sep='\t')
users = data['user'].unique()

In [ ]:
results = defaultdict(list)

with torch.no_grad()
    for user in tqdm(users):

        subset = data[data['user']==user]
        texts = subset['text'].tolist()

        embeddings = [extract_emb(text) for text in texts]
        embeddings = torch.cat(embeddings,dim=0)

        atm, conicity = get_conicity(embeddings)

        results['user'].append(user)
        results['conicity'].append(float(conicity.cpu().detach().numpy()))

In [ ]:
results

In [ ]:
def compute_distinctiveness(vectors):

    distinct = []
    for vec in tqdm(vectors):
        cosines = torch.matmul(vec,vectors.transpose(1,0))
        cosines = cosines.cpu().detach().numpy()
        neighbors = np.sum(np.where(cosines>0.5,1,0))
        distinct.append(neighbors)
        
    distinct = np.array(distinct)
    distinct = 1-distinct/len(distinct)
    return distinct

def extract_emb(text):
    tokenized = tokenizer.encode_plus(text,add_special_tokens=True, max_length=102,truncation=True,return_tensors="pt")
    hidden = model(tokenized['input_ids'],tokenized['attention_mask'])
    hidden = F.normalize(hidden,dim=-1)
    hidden = hidden.cpu().detach().numpy()
    return hidden

In [ ]:
samples = data.groupby('user').sample(1)

In [77]:
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import os
from scipy.stats import spearmanr

In [120]:
con = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/amazon_conicity')

In [121]:
con = con.sort_values('conicity')

length = len(con)

size = int(length*0.33)

random = con.sample(size)

upper = con.iloc[-size:,:]

lower = con.iloc[:size,:]

moderate = con.iloc[size:size*2,:]

In [122]:
data = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/amazon_test_samples',sep='\t')

In [127]:
random_data = data[data['user'].isin(random['user'])]
random_data.to_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/resample/amazon_random_conicity',sep='\t',index=None)

In [128]:
upper_data = data[data['user'].isin(upper['user'])]
upper_data.to_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/resample/amazon_upper_conicity',sep='\t',index=None)

In [129]:
lower_data = data[data['user'].isin(lower['user'])]
lower_data.to_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/resample/amazon_lower_conicity',sep='\t',index=None)

In [130]:
moderate_data = data[data['user'].isin(moderate['user'])]
moderate_data.to_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/resample/amazon_moderate_conicity',sep='\t',index=None)

In [ ]:
path = '/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/resample'
files = os.listdir(path)

In [119]:
files = ['reddit_moderate_distinct','reddit_random_distinct','reddit_upper_distinct','reddit_lower_distinct']
for file in files:
    with open(os.path.join(path,file+"_contrastive"),'w') as out:
        
        data = pd.read_csv(os.path.join(path,file),sep='\t')
        authors = data['user'].unique().tolist()
        for author in tqdm(authors):
            samples = data[data['user']==author]
            instances = samples.sample(6)
            remain = samples.drop(instances.index)
            for i,instance in instances.iterrows():
                pair = remain.sample(1)
                out.write("%s\t\t%s\t\t%s\t\t%s\t\t%s\t\t%s\n"%(instance['user']+'--'+pair['user'].item(),instance['id']+'--'+pair['id'].item(),instance['text'],pair['text'].item(),1,instance['domain']+'--'+pair['domain'].item()))
            
             
            negatives = data[data['user']!=author]
            for k in range(6):
                instance = samples.sample(1)
                negative = negatives.sample(1)
                out.write("%s\t\t%s\t\t%s\t\t%s\t\t%s\t\t%s\n"%(instance['user'].item()+"--"+negative['user'].item(),instance['id'].item()+"--"+negative['id'].item(),instance['text'].item(),negative['text'].item(),0,instance['domain'].item()+"--"+negative['domain'].item()))


 11%|█▏        | 628/5477 [00:16<02:11, 36.86it/s]


 23%|██▎       | 1243/5477 [00:33<01:52, 37.79it/s]


 34%|███▍      | 1855/5477 [00:49<01:37, 37.02it/s]


 45%|████▌     | 2469/5477 [01:05<01:22, 36.60it/s]


 56%|█████▋    | 3081/5477 [01:22<01:04, 37.09it/s]


...

 58%|█████▊    | 3192/5477 [01:27<01:03, 36.16it/s]


 70%|███████   | 3838/5477 [01:44<00:36, 45.29it/s]


 81%|████████▏ | 4456/5477 [02:00<00:28, 36.11it/s]


 92%|█████████▏| 5063/5477 [02:17<00:11, 36.02it/s]


100%|██████████| 5477/5477 [02:28<00:00, 36.78it/s]


In [ ]:
files = ['amazon_moderate_conicity','amazon_random_conicity','amazon_upper_conicity','amazon_lower_conicity']
for file in files:
    with open(os.path.join(path,file+"_contrastive"),'w') as out:
        
        data = pd.read_csv(os.path.join(path,file),sep='\t')
        authors = data['user'].unique().tolist()
        for author in tqdm(authors):
            samples = data[data['user']==author]
            instances = samples.sample(6)
            remain = samples.drop(instances.index)
            for i,instance in instances.iterrows():
                pair = remain.sample(1)
                out.write("%s\t\t%s\t\t%s\t\t%s\t\t%s\t\t%s\n"%(instance['user']+'--'+pair['user'].item(),instance['product']+'--'+pair['product'].item(),instance['text'],pair['text'].item(),1,instance['domain']+'--'+pair['domain'].item()))
            
             
            negatives = data[data['user']!=author]
            for k in range(6):
                instance = samples.sample(1)
                negative = negatives.sample(1)
                out.write("%s\t\t%s\t\t%s\t\t%s\t\t%s\t\t%s\n"%(instance['user'].item()+"--"+negative['user'].item(),instance['product'].item()+"--"+negative['product'].item(),instance['text'].item(),negative['text'].item(),0,instance['domain'].item()+"--"+negative['domain'].item()))

In [92]:
d1 = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/amazon_distinct_0',sep='\t')
d2 = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/amazon_distinct_1',sep='\t')
d3 = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/amazon_distinct_2',sep='\t')
d4 = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/amazon_distinct_3',sep='\t')
d5 = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/amazon_distinct_4',sep='\t')

d1 = d1.sort_values('user')
d2 = d2.sort_values('user')
d3 = d3.sort_values('user')
d4 = d4.sort_values('user')
d5 = d5.sort_values('user')

avg_distinct = (d1['distinct'] + d2['distinct'] + d3['distinct'] + d4['distinct'] + d5['distinct'])/5

d1['avg_d'] = avg_distinct

In [105]:
d1 = d1.sort_values('avg_d')

length = len(d1)

size = int(length*0.33)

random = d1.sample(size)

upper = d1.iloc[-size:,:]

lower = d1.iloc[:size,:]

moderate = d1.iloc[size:size*2,:]

In [106]:
data = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/amazon_test_samples',sep='\t')

In [108]:
random_data = data[data['user'].isin(random['user'])]
random_data.to_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/resample/amazon_random_distinct',sep='\t',index=None)

In [109]:
upper_data = data[data['user'].isin(upper['user'])]
upper_data.to_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/resample/amazon_upper_distinct',sep='\t',index=None)

In [110]:
lower_data = data[data['user'].isin(lower['user'])]
lower_data.to_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/resample/amazon_lower_distinct',sep='\t',index=None)

In [111]:
moderate_data = data[data['user'].isin(moderate['user'])]
moderate_data.to_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/resample/amazon_moderate_distinct',sep='\t',index=None)

In [112]:
d1 = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/reddit_distinct_0',sep='\t')
d2 = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/reddit_distinct_1',sep='\t')
d3 = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/reddit_distinct_2',sep='\t')
d4 = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/reddit_distinct_3',sep='\t')
d5 = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/reddit_distinct_4',sep='\t')

d1 = d1.sort_values('user')
d2 = d2.sort_values('user')
d3 = d3.sort_values('user')
d4 = d4.sort_values('user')
d5 = d5.sort_values('user')

avg_distinct = (d1['distinct'] + d2['distinct'] + d3['distinct'] + d4['distinct'] + d5['distinct'])/5

d1['avg_d'] = avg_distinct

In [113]:
d1 = d1.sort_values('avg_d')

length = len(d1)

size = int(length*0.33)

random = d1.sample(size)

upper = d1.iloc[-size:,:]

lower = d1.iloc[:size,:]

moderate = d1.iloc[size:size*2,:]

In [114]:
data = pd.read_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/reddit_test_samples',sep='\t')

In [115]:
random_data = data[data['user'].isin(random['user'])]
random_data.to_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/resample/reddit_random_distinct',sep='\t',index=None)

In [116]:
upper_data = data[data['user'].isin(upper['user'])]
upper_data.to_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/resample/reddit_upper_distinct',sep='\t',index=None)

In [117]:
lower_data = data[data['user'].isin(lower['user'])]
lower_data.to_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/resample/reddit_lower_distinct',sep='\t',index=None)

In [118]:
moderate_data = data[data['user'].isin(moderate['user'])]
moderate_data.to_csv('/gpfs/accounts/lingjzhu_root/lingjzhu1/lingjzhu/authorship/resample/reddit_moderate_distinct',sep='\t',index=None)